In [1]:
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable
from pyspark.sql import SparkSession

In [3]:
builder = (
    SparkSession
    .builder
    .master("spark://spark:7077")
    .appName("rawApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [4]:
spark.sql("DESCRIBE HISTORY delta.`/opt/workspace/data/raw/iris/`").show()

+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      5|2023-05-27 04:25:...|  null|    null|    WRITE|{mode -> Overwrit...|null|    null|     null|          4|  Serializable|        false|{numFiles -> 1, n...|        null|Apache-Spark/3.3....|
|      4|2023-05-27 04:16:...|  null|    null|    WRITE|{mode -> Overwrit...|null|    null|     null|          3|  Serializable|        false|{numFiles -> 1, n...|        null|Apache-Spark/3.3....|
|      3|2

In [5]:
df = (
    spark
    .read
    .format('delta')
    #.option("readChangeFeed", "true")
    #.option("startingVersion", 5)
    .load('/opt/workspace/data/raw/iris/')
)

In [6]:
df.show()

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
|  6|          5.4|         3.9|          1.7|         0.4|Iris-setosa|
|  7|          4.6|         3.4|          1.4|         0.3|Iris-setosa|
|  8|          5.0|         3.4|          1.5|         0.2|Iris-setosa|
|  9|          4.4|         2.9|          1.4|         0.2|Iris-setosa|
| 10|          4.9|         3.1|          1.5|         0.1|Iris-setosa|
| 11|          5.4|         3.7|          1.5|         0.2|Iris-

In [7]:
df = df.drop_duplicates()

In [8]:
df.write.mode('overwrite').format('delta').save('/opt/workspace/data/cleaned/iris/')

In [9]:
spark.stop()